## swissBUILDINGS3D


In [2]:
import geopandas as gpd
import pandas as pd
import shapely
import geopandas_stac as stac
import geoutils

In [3]:
df = stac.getAssets("ch.swisstopo.swissbuildings3d_2")

In [4]:
list(df['variant'].unique())

['base']

In [5]:
list(df['proj'].unique())

[2056]

In [6]:
list(df['type'].unique())

['application/x.dxf+zip', 'application/x.filegdb+zip']

In [7]:
df_gdb = df.query("type == 'application/x.filegdb+zip'")

In [8]:
df_dxf = df.query("type == 'application/x.dxf+zip'")

### GDB

In [9]:
# Bundeshaus:
lat = 46.94653998135123
lng = 7.444120726365559

point = shapely.geometry.Point(lng, lat)
bundeshaus_punkt = gpd.GeoDataFrame(geometry=gpd.GeoSeries(point, crs="epsg:4326"))
df_bundeshaus = gpd.sjoin(df_gdb, bundeshaus_punkt, op='contains')

In [10]:
df_bundeshaus

,name,type,bbox,created,updated,href,proj,gsd,variant,checksum,geometry,index_right
2861,swissbuildings3d_2_2020-10_2056_5728.gdb.zip,application/x.filegdb+zip,"[5.95590142200007, 45.81795851, 10.492170721, ...",2021-02-16T09:13:38.420521Z,2021-02-16T09:13:38.420538Z,https://data.geo.admin.ch/ch.swisstopo.swissbu...,2056,0,base,122066022d21874ac213c661e2dc502ee46cf4fcc8f111...,"POLYGON ((5.95590 45.81796, 5.95590 47.80845, ...",0
3230,swissbuildings3d_2_2021-05_2056_5728.gdb.zip,application/x.filegdb+zip,"[5.2229406, 45.3204053, 11.2575962, 48.2425816]",2021-06-16T15:18:38.661183Z,2021-07-12T12:00:34.390986Z,https://data.geo.admin.ch/ch.swisstopo.swissbu...,2056,0,base,1220EAFCCB5AF66FCFC02107CF962E2C32725090228670...,"POLYGON ((5.33859 45.35949, 5.22294 48.24258, ...",0


In [11]:
urls = stac.getUrlList(df_bundeshaus)
urls

['https://data.geo.admin.ch/ch.swisstopo.swissbuildings3d_2/swissbuildings3d_2_2020-10/swissbuildings3d_2_2020-10_2056_5728.gdb.zip',
 'https://data.geo.admin.ch/ch.swisstopo.swissbuildings3d_2/swissbuildings3d_2_2021-05/swissbuildings3d_2_2021-05_2056_5728.gdb.zip']

In [12]:
geoutils.download(urls[0], "daten/building.zip", True)

19% done 	[*******************---------------------------------------------------------------------------------]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



54% done 	[******************************************************----------------------------------------------]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



94% done 	[**********************************************************************************************------]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



100% done 	[****************************************************************************************************]


In [ ]:
import zipfile
with zipfile.ZipFile("daten/building.zip", 'r') as zip_ref:
    zip_ref.extractall("daten/b")

In [ ]:
import fiona

gdb = "daten/b/FGDB_LV95LN02/SWISSBUILDINGS3D_2_0_CHLV95LN02.gdb"
fiona.listlayers(gdb);

In [ ]:
_1091_11 = fiona.open(gdb, 'r', layer='_1091_11')

i = iter(_1091_11)

data = next(i)
data

In [ ]:
from shapely.geometry import mapping, shape

geom = shape(data['geometry'])

In [ ]:
geom

In [ ]:
coords = data['geometry']['coordinates']

print(len(coords))

In [ ]:
coords

### Arbeiten mit den 3D Daten

Wir könnten z.B. ein OBJ File daraus machen...

Dies wird hier ohne offset direkt in LV95 gemacht. Es wäre sinnvoller die Koordinatengrössse zu reduzieren


In [ ]:
vertices = ""
faces = ""
cnt = 1

for element in coords:
    for v in element[0]:
        vertices += f"v {v[0]} {v[1]} {v[2]}\n"  # Achtung, wir sollten eigentlich die Höhe in y schreiben
    faces += "f "
    for i in range(0,len(element[0])):
        faces += str(cnt)
        faces += " "
        cnt += 1
    faces += "\n"

print(vertices)
print(faces)
        

### DXF

In [ ]:
# Bundeshaus:
lat = 46.94653998135123
lng = 7.444120726365559

point = shapely.geometry.Point(lng, lat)
bundeshaus_punkt = gpd.GeoDataFrame(geometry=gpd.GeoSeries(point, crs="epsg:4326"))
df_bundeshaus = gpd.sjoin(df_dxf, bundeshaus_punkt, op='contains')

urls = stac.getUrlList(df_bundeshaus)
urls

In [ ]:
geoutils.download(urls[0], "daten/building_dxf.zip", True)

In [ ]:
import zipfile
with zipfile.ZipFile("daten/building_dxf.zip", 'r') as zip_ref:
    zip_ref.extractall("daten/b")